# Analisis para saber si se van a alinear o se pueden alinear adecuadamente los candidatos

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../')
sys.path.append('../math_tricks/')
import funciones_CLICK as fc
from scipy.spatial.distance import pdist, squareform
import multiprocessing
from functools import partial
import warnings
warnings.simplefilter("ignore")

%matplotlib inline

--------------------------------------------
__Encontramos que cuando se rota y traslada [referencia](https://academic.oup.com/nar/article/45/1/e5/2871113), hay que calcular la distancia de UN residuo $A_i$ VS todos los $B_j$ y quedarnos con la pareja $(A_i,B_j)$ tal que su distancia sea menor a 3.5 A y si hay mas de uno te quedas con el menor!!__

## Encontrar matcheo de residuo a residuo, en lo que calcula la rotacion y traslacion.

In [5]:
new_df_cliques1 = pd.read_pickle('../clique1.pkl')
new_df_cliques2 = pd.read_pickle('../clique2.pkl')

candidatos = pd.read_csv('../candidatos.csv')

df_atoms1 = pd.read_pickle('../clique1_df_atributos.pkl')
df_atoms2 = pd.read_pickle('../clique2_df_atributos.pkl')

rmsd = pd.read_pickle("../rmsd_picke.pkl").reset_index(drop=True)

In [21]:
%%time
a_1 = df_atoms1[df_atoms1.atom_name == 'CA'].vector.values

lista_vectores_gorro = []
for bari in new_df_cliques1.baricentro_clique.values:
    lista_pre_vectores = []

    for coord in a_1:
        lista_pre_vectores.append(coord - bari)

    lista_vectores_gorro.append(lista_pre_vectores)

CPU times: user 41 ms, sys: 44 µs, total: 41.1 ms
Wall time: 39.6 ms


In [22]:
vectores_gorro_proteina_1 = pd.DataFrame(lista_vectores_gorro)

In [23]:
%%time
a_2 = df_atoms2[df_atoms2.atom_name == 'CA'].vector.values

lista_vectores_gorro = []
for bari in new_df_cliques2.baricentro_clique.values:
    lista_pre_vectores = []
    for coord in a_2:
        lista_pre_vectores.append(coord - bari)

    lista_vectores_gorro.append(lista_pre_vectores)

CPU times: user 25.4 ms, sys: 39 µs, total: 25.5 ms
Wall time: 24.7 ms


In [24]:
vectores_gorro_proteina_2 = pd.DataFrame(lista_vectores_gorro)

In [26]:
vectores_gorro_proteina_1.shape

(553, 71)

In [27]:
for i,j in enumerate(rmsd.columns):
    print(i,j)

0 rmsd
1 candidatos
2 matriz_rotacion
3 candidato_clique_1
4 candidato_clique_2


In [28]:
for i,j in enumerate(new_df_cliques1.columns):
    print(i,j)

0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 ss_0
8 ss_1
9 ss_2
10 ss_3
11 ss_4
12 ss_5
13 ss_6
14 coord_clique_0
15 coord_clique_1
16 coord_clique_2
17 coord_clique_3
18 coord_clique_4
19 coord_clique_5
20 coord_clique_6
21 matriz_coordenadas
22 baricentro_clique
23 vec_gorro_0
24 vec_gorro_1
25 vec_gorro_2
26 vec_gorro_3
27 vec_gorro_4
28 vec_gorro_5
29 vec_gorro_6
30 vectores_gorro
31 distancia_promedio


In [36]:
# idx = id_so = 354
# so = 0.0
idx = 0
df_rmsd = rmsd.values
df_new_df_clique1 = new_df_cliques1.values
df_new_df_clique2 = new_df_cliques2.values
df_vectores_gorro_proteina_1 = vectores_gorro_proteina_1.values
df_vectores_gorro_proteina_2 = vectores_gorro_proteina_2.values

atoms1 = df_atoms1.atom_number.values
atoms2 = df_atoms2.atom_number.values
number_of_residues = df_atoms1.shape[0]

protein_to_compare = np.array([i for i in df_atoms2.vector.values])

if so == 1.0:
    id_so = idx
    print(so, id_so)
    print('////' * 20)


clique_in_protein_1 = df_rmsd[idx, 3]  # se extrae la info de los candidatos
clique_in_protein_2 = df_rmsd[idx, 4]

# se obtienen el numero de residuo de los candidatos
los_del_clique_1 = df_new_df_clique1[clique_in_protein_1, [0, 1, 2, 3, 4, 5, 6]]
los_del_clique_2 = df_new_df_clique1[clique_in_protein_2, [0, 1, 2, 3, 4, 5, 6]]

matriz_rotacion = df_rmsd[idx, 2]  # se obtiene la matriz de rotacion de los cliques candidatos

# se obtienen los vectores gorro de la proteina a rotar y trasladar
vector_gorro = df_vectores_gorro_proteina_1[clique_in_protein_1]
# se aplica la rotacion a vectores gorro
coord_vectores_rotados = [np.matmul(matriz_rotacion, i.reshape(3, 1)).T[0] for i in vector_gorro]
# se obtiene el baricentro de la proteina 2
baricentro_proteina_2 = df_new_df_clique2[clique_in_protein_2, 22]
# se suma el baricentro a vectores rotados de la proteina 1
protein_trasladado_rotado = coord_vectores_rotados + baricentro_proteina_2  # nuevas coordendas proteina 1
# se quitan residuos que pertenezcan al clique candidato para calcular la distancia y emparejar al mejor.
protein_trasladado_rotado_sin_clique = np.delete(protein_trasladado_rotado, los_del_clique_1, 0)
protein_to_compare_sin_clique = np.delete(protein_to_compare, los_del_clique_2, 0)
# numeros de atomos
atom_numbers1 = np.delete(atoms1, los_del_clique_1, 0)
atom_numbers2 = np.delete(atoms2, los_del_clique_2, 0)

residuos_match = []  # aqui se guardan las parejas de residuos

# se itera por cada residuo ya rotado y trasladado
for residuo_not_in_clique, res_1 in zip(protein_trasladado_rotado_sin_clique, atom_numbers1):
          for residuo, res_2 in zip(protein_to_compare_sin_clique, atom_numbers2):
            distancia = pdist(np.array([residuo_not_in_clique, residuo]), metric='euclidean').item()
            if distancia < 3.5:
                print('a')
#                 list_cand =[pdist(np.array([residuo_not_in_clique, res_ngb.coord]), metric='euclidean').item()
#                      for res_ngb in residuo.ngb]
#                 break
#                 residuos_match.append([distancia, (res_1, res_2)])

df = pd.DataFrame(residuos_match, columns=['distancia', 'candigatos'])


a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a


In [47]:
id_so = 0
so = 0.0

def align_residues(idx, so):

    df_rmsd = rmsd.values
    df_new_df_clique1 = new_df_cliques1.values
    df_new_df_clique2 = new_df_cliques2.values
    df_vectores_gorro_proteina_1 = vectores_gorro_proteina_1.values
    df_vectores_gorro_proteina_2 = vectores_gorro_proteina_2.values

    atoms1 = df_atoms1[df_atoms1.atom_name == 'CA'].atom_number.values
    atoms2 = df_atoms2[df_atoms1.atom_name == 'CA'].atom_number.values
    number_of_residues = df_atoms1[df_atoms1.atom_name == 'CA'].shape[0]
    
    protein_to_compare = np.array([i for i in df_atoms2[df_atoms2.atom_name == 'CA'].vector.values])
    
    clique_in_protein_1 = df_rmsd[idx, 3]  # se extrae la info de los candidatos
    clique_in_protein_2 = df_rmsd[idx, 4]

    # se obtienen el numero de residuo de los candidatos
    los_del_clique_1 = df_new_df_clique1[clique_in_protein_1, [0, 1, 2, 3, 4, 5, 6]]
    los_del_clique_2 = df_new_df_clique1[clique_in_protein_2, [0, 1, 2, 3, 4, 5, 6]]

    matriz_rotacion = df_rmsd[idx, 2]  # se obtiene la matriz de rotacion de los cliques candidatos

    # se obtienen los vectores gorro de la proteina a rotar y trasladar
    vector_gorro = df_vectores_gorro_proteina_1[clique_in_protein_1]
    # se aplica la rotacion a vectores gorro
    coord_vectores_rotados = [np.matmul(matriz_rotacion, i.reshape(3, 1)).T[0] for i in vector_gorro]
    # se obtiene el baricentro de la proteina 2
    baricentro_proteina_2 = df_new_df_clique2[clique_in_protein_2, 22]
    # se suma el baricentro a vectores rotados de la proteina 1
    protein_trasladado_rotado = coord_vectores_rotados + baricentro_proteina_2  # nuevas coordendas proteina 1
    # se quitan residuos que pertenezcan al clique candidato para calcular la distancia y emparejar al mejor.
    protein_trasladado_rotado_sin_clique = np.delete(protein_trasladado_rotado, los_del_clique_1, 0)
    protein_to_compare_sin_clique = np.delete(protein_to_compare, los_del_clique_2, 0)
    # numeros de atomos
    atom_numbers1 = np.delete(atoms1, los_del_clique_1, 0)
    atom_numbers2 = np.delete(atoms2, los_del_clique_2, 0)

    residuos_match = []  # aqui se guardan las parejas de residuos

    # se itera por cada residuo ya rotado y trasladado
 
    for residuo_not_in_clique, res_1 in zip(protein_trasladado_rotado_sin_clique, atom_numbers1):
        for residuo, res_2 in zip(protein_to_compare_sin_clique, atom_numbers2):
            distancia = pdist(np.array([residuo_not_in_clique, residuo]), metric='euclidean').item()
            if distancia < 3.5:
                residuos_match.append([distancia, (res_1, res_2)])

    df = pd.DataFrame(residuos_match, columns=['distancia', 'candigatos'])
    df = df.sort_values('distancia')
    df['cand1'] = df.candigatos.str[0]
    df['cand2'] = df.candigatos.str[1]

    c1 = []
    c2 = []
    cand_n = []
    for i in df.values:

        if (i[2] in c1) or (i[3] in c2) or (i[0] > 3.5):
            continue
        else:
            c1.append(i[2])
            c2.append(i[3])
            cand_n.append(i)

    so_temp = len(cand_n) / number_of_residues

    return idx, so_temp

In [52]:
id_so = 0
so = 0.0
for i in np.arange(len(df_rmsd))[:400]:
    id_so, so_temp = (align_residues(i,so))
    if so <= so_temp:
        so = so_temp
        id_so = i
        print(id_so,so)
    if so == 1:
        break

0 0.3380281690140845
1 0.647887323943662
248 0.7605633802816901
249 0.7605633802816901
250 1.0


In [30]:
%%time
align_residues(143,0)

CPU times: user 97.9 ms, sys: 4.04 ms, total: 102 ms
Wall time: 98.6 ms


143

In [31]:
((0.80 * 33740)/60)/60

7.497777777777778

In [32]:
((0.80 * 500)/60)

6.666666666666667

In [31]:
# p = multiprocessing.Pool(multiprocessing.cpu_count()-1)

# id_so = p.map(partial(align_residues, so=so), np.arange(len(df_rmsd))[:100])

# p.close()
# p.join()

# print(id_so)